# Setup Minedojo Environment

In [ ]:
from Environments import SkyRunner, MultithreadGym
frame_stack = False
frames = 4
use_grayscale=False

# Multithreaded environment wrapper
env = MultithreadGym.MultithreadGym(thread_int=1, env_int=1, frame_stack=frame_stack, frames_int=frames, use_grayscale=use_grayscale)

# Evaluation environment
eval_env = SkyRunner.CustomEnv(frame_stack=frame_stack, frames_int=frames, use_grayscale=use_grayscale)
eval_env.reset()

# Begin training

In [ ]:
import train_openAI
import importlib

importlib.reload(train_openAI)

train_openAI.train(env, eval_env=eval_env, eval_freq=5000, use_prioritized_replay=False, name="baseline-4")

# Shutdown Environments

In [ ]:
env.close()
eval_env.close()

# Load existing model

In [ ]:
from stable_baselines3 import DQN
from CustomBaselines3.DoubleDQN import DoubleDQN

_env = env
model = DoubleDQN.load("/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/dDQN-checkpoints/checkpoint_30000_steps.zip")

obs = env.reset()
acc_r = 0
while True:
    act, st = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(act)

    acc_r += reward

    env.render()

    if done:
        obs = env.reset()
        print("Finished with reward %d" % acc_r)
        acc_r = 0